<a href="https://colab.research.google.com/github/boriskuchin/MADMO-BASE-2024/blob/main/02extra_magic_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Декораторы

Декораторы можно применять не только к обычным функциям, но и к декораторам. Помимо декораторов, созданных пользователями, в Python имеется несколько встроенных декораторов.

## `@staticmethod`

Рассмотрим пример - создаем класс `Calendar`, который хранит в себе словарь `"событие": дата`:

In [ ]:
class Calendar:
    def __init__(self):
        self.events = dict()

    def add_event(self, event, date):
        self.events[event] = date

Еще для календаря было бы полезно проверять является ли та или иная дата выходным днем, т.е. иметь метод, который принимает на вход дату и возвращает `True`, если дата выпадает на сб или вс:

In [ ]:
class Calendar:
    def __init__(self):
        self.events = {}

    def add_event(self, event, date):
        self.events[event] = date

    def is_weekend(self, date):
        return date.weekday() > 4  # Пн Вт Ср Чт Пт Сб Вс

In [ ]:
import datetime

today = datetime.date.today()
temp_cal = Calendar()
Calendar.is_weekend(temp_cal, today)


False

Заметим, что в методе `is_weekend` никак не используется переданный через self экземпляр класса. Это связано с тем, что является ли тот или иной день выходным не зависит от конкретного календаря.

Было бы еще приятно, если бы была возможность вызывать метод is_weekend от имени класса, а не экземпляра класса. Эту возможность нам предоставляет декоратор `@staticmethod`:

In [ ]:
class Calendar:
    def __init__(self):
        self.events = {}

    def add_event(self, event, date):
        self.events[event] = date

    @staticmethod
    def is_weekend(date):  # Теперь не указываем self - он не нужен
        return date.weekday() > 4  # Пн Вт Ср Чт Пт Сб Вс

In [ ]:
import datetime

cal = Calendar()
today = datetime.date.today()
cal.add_event("webinar", today)

In [ ]:
cal.is_weekend(today)

False

In [ ]:
Calendar.is_weekend(today)

False

Здесь мы используем модуль datetime, она достаточно простая, поэтому оставляю [ссылку](https://pythonru.com/primery/kak-ispolzovat-modul-datetime-v-python).

In [ ]:
today.year

2022

**Вопрос:** почему бы просто не вынести статический метод вне класса как функцию?

In [ ]:
def is_weekend(date):
    return date.weekday() > 4  # Пн Вт Ср Чт Пт Сб Вс


class Calendar:
    def __init__(self):
        self.events = {}

    def add_event(self, event, date):
        self.events[event] = date

В некоторых случаях так и следует делать - если этот метод пригодится и другим классам. Но если этот метод используется только для данного класса, то лучше его объявить как статический метод.

## `@classmethod`

Предположим, что хотим прочитать сохраненный календарь из файла и записать его в переменную. Создадим для этого метод `from_json`. Нам точно не нужен существующий экземпляр класса, чтобы создать новый, поэтому попробуем объявить этот метод как статический:

In [ ]:
class Calendar:
    def __init__(self):
        self.events = {}

    def add_event(self, event, date):
        self.events[event] = date

    @staticmethod
    def is_weekend(date):
        return date.weekday() > 4

    @staticmethod
    def from_json(filename):
        cal = Calendar()
        # Считываем данные из файла и помещаем в cal
        return cal

In [ ]:
cal = Calendar.from_json("my_cal.json")
type(cal)

__main__.Calendar

Вроде все работает как надо. Но рассмотрим ситуацию - мы создаем дочерний класс WorkCalendar и создаем его экземпляр с помощью родительского метода `from_json`:

In [ ]:
class WorkCalendar(Calendar):
    pass

In [ ]:
workcal = WorkCalendar.from_json("my_calendat.cal")
type(workcal)

__main__.Calendar

In [ ]:
class Calendar:
    def __init__(self):
        self.events = {}

    def add_event(self, event, date):
        self.events[event] = date

    @staticmethod
    def is_weekend(date):
        return date.weekday() > 4

    def from_json(self, filename):
        cal = self.__class__()
        #cal = Calendar()
        # Считываем данные из файла и помещаем в cal
        return cal

In [ ]:
class WorkCalendar(Calendar):
    pass

In [ ]:
workcal = WorkCalendar.from_json(WorkCalendar(),"my_calendar.cal")
type(workcal)

__main__.WorkCalendar

В результате получили экземпляр класса `Calendar`, а не `WorkCalendar`. В таких ситуациях нам пригодится декоратор `@classmethod`:

In [ ]:
class Calendar:
    def __init__(self):
        self.events = {}

    def add_event(self, event, date):
        self.events[event] = date

    @staticmethod
    def is_weekend(date):
        return date.weekday() > 4

    @classmethod
    def from_json(cls, filename):
        cal = cls()
        # Считываем данные из файла и помещаем в cal
        return cal

In [ ]:
class WorkCalendar(Calendar):
    pass

In [ ]:
workcal = WorkCalendar.from_json("my_calendar.cal")
type(workcal)

__main__.WorkCalendar

# Специальные методы классов

Специальные методы классов (aka магические методы, dundered-методы). С одним из них мы уже знакомились - метод `__init__` - конструктор экземпляра.

Сегодня познакомимся и с остальными magic-методами.

## `__str__`

Создадим простой класс: вектор с двумя координатами и цветом.

![vector](https://lh4.googleusercontent.com/rF9T2gg0ykY-kmJAcWsAmEkfKIsdSwx79F8cpAJrZmlJlE_Q8sCHRc0ENSPkyJyfkPx-W_2BD1lWpFE6fw_bEgfvRTUUHVs-_47sjFa23s-4EzaMZSE_Q2CwWXB5xSSy1Bg2cRTY)

In [ ]:
import random

class Vector:
    def __init__(self, x=0, y=0, color=None):
        print("initializing a vector")
        if type(x) is not int or type(y) is not int:
            raise AttributeError('x and y should be int')

        self._x = x
        self._y = y
        self._color = color

    def get_x(self):
        return self._x

    def get_y(self):
        return self._y

Создадим экземпляр вектора и посмотрим на его строчное представление:

In [ ]:
vector = Vector(1, 2, 'red')
str(vector)

initializing a vector


'<__main__.Vector object at 0x7f5c1dea9650>'

In [ ]:
print(vector)

Информативно, но не очень красиво. Можем переопределить поведение метода приведения нашего класса к строковому типу - `__str__`:

In [ ]:
class VectorWithStr(Vector):
    def __str__(self):
        #return 'vector ({}, {}) of color {}'.format(self._x, self._y, self._color)
        return f"vector ({self._x}, {self._y}) of color {self._color}"
        #return "vector ("+str(self._x)+", "+str(self._y)+") of color "+str(self._color)

In [ ]:
vector = VectorWithStr(1, 2, 'red')
str(vector)

initializing a vector


'vector (1, 2) of color red'

Просто преобразование в строку? Конечно, нет. Неявные преобразования иногда происходят там, где мы их не ожидаем, например, при вызове `print`:

In [ ]:
print(vector)

vector (1, 2) of color red


## `__repr__`

Посмотрим как объект будет выглядеть в качестве ключа для словаря:

In [ ]:
mydict = {}
mydict[vector]

KeyError: ignored

Аналогично и при выводе списка, содержащего наш объект:

In [ ]:
mylist = [vector]
print(mylist)

Почему опять "некрасивые" строки?! В Python используется два способа приведения к строке. Это функции `str` и `repr`, которые отличаются своим назначением.

- `str` используется там, где нужна человекочитаемость
- `repr` реализуется так, чтобы можно было однозначно определить, о каком объекте идет речь, вызывается явно

Если `repr` не реализован, используется стандартный вариант, а если не реализован `str`, то вместо него используется `repr`.

Добавим `repr`:

In [ ]:
class VectorWithRepr(Vector):
    def __repr__(self):
        return 'vector representation (x: {}, y: {}, color: {})'.format(self._x, self._y, self._color)

In [ ]:
vector = VectorWithRepr(1, 2, 'red')
vector

initializing a vector


vector representation (x: 1, y: 2, color: red)

In [ ]:
mylist = [vector]
mylist

[vector representation (x: 1, y: 2, color: red)]

In [ ]:
mydict = {}
mydict[vector]

KeyError: ignored

Создадим класс с обоими реализованными методами:

In [ ]:
class VectorWithBothReprAndStr(VectorWithRepr, VectorWithStr):
    pass

In [ ]:
vector = VectorWithBothReprAndStr(1, 2, 'red')
list_with_vector = [vector]
# вот здесь должны получиться разные значения
print(vector)
print(list_with_vector)
print(type(list_with_vector[0]), list_with_vector[0]._x)

initializing a vector
vector (1, 2) of color red
[vector representation (x: 1, y: 2, color: red)]
<class '__main__.VectorWithBothReprAndStr'> 1


## Арифметические методы

В Python имеются magic-методы для поддержки арифметических операций с пользовательскими классами:

In [ ]:
import math
import random

class VectorWithMath(VectorWithBothReprAndStr):
    def __abs__(self):
        return math.hypot(self._x, self._y)

    def __add__(self, other):
        return VectorWithMath(
            self.get_x() + other.get_x(),
            self.get_y() + other.get_y(),
            random.choice((str(self._color), str(other._color)))
        )

    def __sub__(self, other):
        return VectorWithMath(
            self.get_x() - other.get_x(),
            self.get_y() - other.get_y(),
            random.choice((str(self._color), str(other._color)))
        )

    # ещё есть div, mul и многое другое

Создадим два объекта и попробуем операции с ними:

In [ ]:
vector1 = VectorWithMath(3, 4, 'blue')
vector2 = VectorWithMath(1, 2, 'red')
vector3 = VectorWithMath(1, 0, 'green')
print(vector1, vector2, vector3)

initializing a vector
initializing a vector
initializing a vector
vector (3, 4) of color blue vector (1, 2) of color red vector (1, 0) of color green


In [ ]:
abs(vector1)

5.0

In [ ]:
vector1 + vector2

initializing a vector


vector representation (x: 4, y: 6, color: red)

In [ ]:
vector1 - vector2

initializing a vector


vector representation (x: 2, y: 2, color: red)

In [ ]:
vector1 + (vector2 + vector3)

initializing a vector
initializing a vector


vector representation (x: 5, y: 6, color: blue)

In [ ]:
vector1.__add__(vector2.__add__(vector3))

initializing a vector
initializing a vector


vector representation (x: 5, y: 6, color: blue)

In [ ]:
vector1 += vector2
vector1

initializing a vector


vector representation (x: 4, y: 6, color: red)

In [ ]:
vector1 + 3

AttributeError: ignored

## Приведение типов

Для преобразования объекта в базовые типы данных имеются соответсвующие magic-методы: `str`, `int`, `float`, `bool`. Добавим преобразование к базовым типам для нашего класса:

In [ ]:
import math

class VectorWithTypes(VectorWithMath):
    def __bool__(self):  # неявно вызывается при использовании объекта в условиях
        return bool(self._x) or bool(self._y)  # True если вектор ненулевой

    def __float__(self):
        return abs(self)  # можем пользоваться built-in функциями, которые будут вызывать методы
        #return self.__abs__()

    def __int__(self):
        return int(float(self))

Проверим работу приведения к типам:

In [ ]:
vector = VectorWithTypes(3, 4, 'blue')
vector

initializing a vector


vector representation (x: 3, y: 4, color: blue)

In [ ]:
int(vector)

5

In [ ]:
float(vector)

5.0

In [ ]:
"vector ~ True" if vector else "vector ~ False"

'vector ~ True'

In [ ]:
vector = VectorWithTypes()
vector

initializing a vector


vector representation (x: 0, y: 0, color: None)

In [ ]:
"vector ~ True" if vector else "vector ~ False"

'vector ~ False'

## Итерирование

Есть два способа сделать объект "итерабельным", т.е. по объекту можно было итерироваться (например, `for .. in ..`):
- методы `__next__` и `__iter__`
- метод `__getitem__` - доступ по индексу

Также есть методы `len` и `reversed` для работы соответствующих встроенных функций.

In [ ]:
class VectorIterable(VectorWithTypes):
    def __getitem__(self, position):
        #if position == 0:
        #    return self._x
        #elif position == 1:
        #    return self._y
        #else:
        #    raise IndexError
        return (self._x, self._y)[position]

    def __len__(self):
        # return 2
        return len((self._x, self._y))  # 2

    def __reversed__(self):
        return (self._x, self._y)[::-1]

Посмотрим как оно работает:

In [ ]:
vector = VectorIterable(100, 500)
vector

initializing a vector


vector representation (x: 100, y: 500, color: None)

In [ ]:
vector[0], vector[1]

(100, 500)

In [ ]:
vector[:1]

(100,)

In [ ]:
reversed(vector)

(500, 100)

In [ ]:
len(vector)

2

Теперь можем использовать объект в цикле `for` - он будет вызывать индексы от 0 до тех пор, пока не вылезет ошибка:

In [ ]:
for coordinate in vector:
    print(coordinate)

100
500


**Замечание** - цикл `for` вызывает `__getitem__` ТОЛЬКО если у класса отсутствует `__iter__`.

Проверим это, сделав наш объект iterable:

In [ ]:
class VectorIterable1(VectorWithTypes):
    def __getitem__(self, position):
        return (self._y, self._x)[position]  # поменяем координаты, чтобы отличить два варианта

    def __iter__(self):
        return iter((self._x, self._y))

    def __len__(self):
        return 2

    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [ ]:
vect = VectorIterable1(3, 5)

for c in vect:
    print(c)

initializing a vector
3
5


In [ ]:
example_set = set()
example_set.add(10)
example_set.add(15)

for item in example_set:
    print(item)

10
15


In [ ]:
example_set[0]

TypeError: ignored

Значит вызывается `__iter__`!

In [ ]:
class NewVector1:
    def __init__(self, *args, color=None):
        self.coords = args

    def __len__(self):
        return len(self.coords)

class NewVector2:
    def __init__(self, color=None, **kwargs):
        self.coords = kwargs

    def __len__(self):
        return len(self.coords.keys())

In [ ]:
test = NewVector1(1,2,3,4,5,6, color='violet')
len(test)

6

In [ ]:
test = NewVector2(x1=1,x2=2,x3=3,x4=4, color='violet')
len(test)

4

## Динамическая работа с атрибутами

В Python существуют 4 magic-метода, которые переопределяют работу с атрибутами:
- `__getattr__` - вызывается при запросе несуществующих атрибутов, аргумент - название атрибута
- `__getattribute__` - вызывается при запросе любых атрибутов
- `__setattr__` - вызывается при изменении значения атрибута (не только существующего)
- `__delattr__` - вызывается при удалении атрибута

Насколько вы помните, в Python нет никакой защиты от "взлома". Попробуем сделать ее самостоятельно!

In [ ]:
class VectorWithAllAttributes(VectorIterable):
    def __getattr__(self, attr_name):
        return "value of {}".format(attr_name)

    def __setattr__(self, attr_name, attr_value):
        if attr_name not in ('_x', '_y', '_color'):
            raise Exception('you shall not add new attributes here, young padawan!')  # Запрещаем добавление атрибутов
        else:
            super().__setattr__(attr_name, attr_value)
            #self.__setattr__(attr_name, attr_value)

    def __delattr__(self, attr_name):
        print('Heh, you can delete nothing')

**Замечание** - важно вызывать `__setattr__` для предка, а не для самого объекта, чтобы не свалиться в рекурсию; если ни от кого не наследовались, можем вызвать `object.__setattr__(self, attr_name, attr_value)`

Создадим объект и посмотрим как он ведет себя:

In [ ]:
vector = VectorWithAllAttributes(1, 2, 'violet')

initializing a vector


In [ ]:
vector.some_attribute

'value of some_attribute'

In [ ]:
vector._color

'violet'

In [ ]:
vector.get_x()

1

In [ ]:
del vector._color

Heh, you can delete nothing


In [ ]:
vector._color

'violet'

In [ ]:
vector.new_attribute = "value"

Exception: ignored

In [ ]:
vector._color = 'gray'

### `__getattr__` vs. `__getattribute__`

Еще один шанс свалиться в бесконечную рекурсию - `__getattribute__`. Поэтому так же нужна устройчивая конструкция:

In [ ]:
class GetAttr:
    attr1 = 1

    def __init__(self):
        self.attr2 = 2

    def __getattr__(self, attr):   # Только для неопределенных атрибутов
        print('get: ' + attr)      # Не attr1: наследуется от класса
        return 3                   # Не attr2: хранится в экземпляре


class GetAttribute:
    attr1 = 1

    def __init__(self):
        self.attr2 = 2

    def __getattribute__(self, attr):  # Вызывается всеми операциями чтения
        print('get: ' + attr)          # Для предотвращения зацикливания используется суперкласс
        if attr == 'attr3':
            return 3
        else:
            return super().__getattribute__(attr)

Посмотрим на их поведение:

In [ ]:
X = GetAttr()

In [ ]:
X.attr1

1

In [ ]:
X.attr2

2

In [ ]:
X.attr3

get: attr3


3

In [ ]:
X.attr4

get: attr4


3

А теперь `GetAttribute()`:

In [ ]:
X = GetAttribute()

In [ ]:
X.attr1

get: attr1


1

In [ ]:
X.attr2

get: attr2


2

In [ ]:
X.attr3

get: attr3


3

In [ ]:
X.attr4

get: attr4


AttributeError: ignored

Extra

In [ ]:
class GetAttr2:
    attr1 = 1

    def __init__(self):
        self.attr2 = 2

    def __getattribute__(self, attr):
        print('get: ' + attr)
        if attr == 'attr3':
            return 3
        else:
            return super().__getattribute__(attr)

In [ ]:
getattr2 = GetAttr2()

In [ ]:
getattr2.attr1

get: attr1


1

In [ ]:
getattr2.attr2

get: attr2


2

In [ ]:
getattr2.attr3

get: attr3


3

In [ ]:
print(dir(getattr2))

get: __dict__
get: __class__
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attr1', 'attr2']


In [ ]:
getattr2.attr4

get: attr4


AttributeError: ignored

## Контексты

В Python есть конструкция, называемая менеджер контекста:

```python
with smth as smth:
    <тело>
```

Она позволяет **гарантированно** выполнять набор команд при начале и при завершении выполнения команд внутри себя.

Частый пример - чтение файла.

Создадим пустой файл:

In [ ]:
%%bash
touch test.txt

Откроем этот файл и запишем туда строку:

In [ ]:
f = open("test.txt", "w")

In [ ]:
type(f)

_io.TextIOWrapper

In [ ]:
f.write("hello, world!")

13

Посмотрим, что записалось в файл:

In [ ]:
%%bash
cat test.txt

Пусто! Такое поведение связано с особенностями работы операционных систем. Поэтому надо закрывать файл после завершения работы с ним!

In [ ]:
f.close()

In [ ]:
%%bash
cat test.txt

hello, world!

Чтобы случайно не забыть закрыть файл после завершения работы с ним, удобно использовать с функцией `open` оператор `with`:

In [ ]:
with open("test.txt", "a") as f:
    f.write("hello, world2!")

In [ ]:
%%bash
cat test.txt

hello, world!hello, world2!

Разберем как создавать собстенные менеджеры контекстов:

Для работы с контекстами есть два magic-метода:

- `__enter__` - выполняется до отработки тела контекста
- `__exit__`  - после обработки тела контекста + ловит ошибку

In [ ]:
class VectorWithContextManager:
    def __enter__(self):
        print('entering context')

    def __exit__(self, exception_class, exception_obj, tb_obj):
        print(exception_class, exception_obj, tb_obj)
        print(dir(tb_obj), tb_obj.tb_lineno)
        print('leaving context')

        return False # -- бросаем ошибку дальше
        #return True  # -- НЕ бросаем ошибку дальше

Проверим как наш менеджер контекста обработает возникающие ошибки:

In [ ]:
with VectorWithContextManager() as vec:  # vec = VectorWithContextManager()
    for i in range(3):
        print(i)
    raise KeyError('something happened inside!')

    for i in range(3, 5):
        print(i)

print('we are out of the context')

entering context
0
1
2
<class 'KeyError'> 'something happened inside!' <traceback object at 0x7f5c1d3c0730>
['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next'] 4
leaving context


KeyError: ignored

In [ ]:
raise KeyError('something happened inside!')
print('A')

KeyError: ignored

In [ ]:
type(vec)

NoneType

In [ ]:
f = open("test.txt", "a")

In [ ]:
print(dir(f))

['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'reconfigure', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'write_through', 'writelines']


Но создавать менеджеры контекстов можно и еще проще - с помощью декоратора `contextmanager` И генераторной функции:

In [ ]:
from contextlib import contextmanager

@contextmanager
def vector_mgr():
    print('handling entering the context')
    yield Vector()
    print('handling leaving the context')

print('statement before context')
with vector_mgr() as vector:
    for i in range(3):
        print(vector)
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


## Создание и удаление объектов

При создании и удалении объектов так же используются magic-методы `__new__` и `__del__`:
- `__new__` - вызывается при создании объекта (до конструктора-инициализации)
- `__del__` - вызывается при удалении объекта

In [ ]:
class VectorInitialized(Vector):
    def __new__(cls, *args, **kwargs):  # метод класса, принимает класс и аргументы конструктора
        print('invoking __new__ method')
        print(cls, args, kwargs)
        return object.__new__(cls)

    def __del__(self):
        print('deleting an object')
        raise Exception("exception while destructing")

In [ ]:
vect = VectorInitialized(1, 2, color='navy blue')
print(vect)

invoking __new__ method
<class '__main__.VectorInitialized'> (1, 2) {'color': 'navy blue'}
initializing a vector


In [ ]:
vect._x

1

In [ ]:
del vect

deleting an object


Exception ignored in: <function VectorInitialized.__del__ at 0x7f5c1d28b9e0>
Traceback (most recent call last):
  File "<ipython-input-102-22db342b9e0d>", line 9, in __del__
Exception: exception while destructing


In [ ]:
vect

NameError: ignored

In [ ]:
tuple(['a','b','c'])

('a', 'b', 'c')

In [ ]:
class NewTuple(tuple):
    def __new__(cls, a):
        return super().__new__(cls, (item.upper() for item in a))

    #def __init__(self, args):
        #for i in range(len(args)):
            #print(self[i])
            #self[i] = self[i].upper()

In [ ]:
NewTuple(['a','b','c'])

('A', 'B', 'C')

### Задачка

Как с помощью метода `__new__` сделать класс "синглтоном" -- объектом, который создается один раз, а при попытке повторного создания возвращается уже готовый объект?

In [ ]:
class SingletonClass:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = object.__new__(cls, *args, **kwargs)
        return cls._instance

In [ ]:
obj1 = SingletonClass()
obj2 = SingletonClass()
assert id(obj1) == id(obj2)
print(id(obj1) == id(obj2))

True


## Callable-объекты

Можно сделать экземпляры класса вызываемыми как функции (чтобы в скобках можно было бы указать аргументы и получить значение).

In [ ]:
class Adder:
    def __init__(self, x):
        self.x = x

    def __call__(self, y):
        return self.x + y

    def call(self, y):
        return self.x + y

In [ ]:
adder = Adder(10)

In [ ]:
print(adder(14))
print(adder.__call__(14))
print(adder.call(14))

24
24
24


In [ ]:
adder.x = sum(i ** 2 for i in range(3))

print(adder(0))
print(adder(0))

5
5


# Extra

## Атрибуты функции

In [ ]:
print?

In [ ]:
print(print.__doc__)
print(type(print.__doc__))

print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)

Prints the values to a stream, or to sys.stdout by default.
Optional keyword arguments:
file:  a file-like object (stream); defaults to the current sys.stdout.
sep:   string inserted between values, default a space.
end:   string appended after the last value, default a newline.
flush: whether to forcibly flush the stream.
<class 'str'>


In [ ]:
def foo(*args, **kwargs):
    'Function which prints arguments.'
    print('args =', args, 'kwargs =', kwargs)

print(*dir(foo), sep=' ')
print(foo.__name__)
print(foo.__doc__) # documentation
print(foo.__module__)

__annotations__ __call__ __class__ __closure__ __code__ __defaults__ __delattr__ __dict__ __dir__ __doc__ __eq__ __format__ __ge__ __get__ __getattribute__ __globals__ __gt__ __hash__ __init__ __init_subclass__ __kwdefaults__ __le__ __lt__ __module__ __name__ __ne__ __new__ __qualname__ __reduce__ __reduce_ex__ __repr__ __setattr__ __sizeof__ __str__ __subclasshook__
foo
Function which prints arguments.
__main__


#### Аттрибуты можно использовать как статические переменные

In [ ]:
def get_next_id():
    if not hasattr(get_next_id, 'value'):
        get_next_id.value = 0

    get_next_id.value += 1
    return get_next_id.value

print(get_next_id())
print(get_next_id())
print(get_next_id())
print('get_next_id.value =', get_next_id.value)

1
2
3
get_next_id.value = 3


#### Где хранятся аргументы по умолчанию?

In [ ]:
def foo(a = 'Hello', b = 1):
    print(a, b)

print('Defaults: ', foo.__defaults__)
foo()

foo.__defaults__ = ('Hello', 'world!')
print('Defaults: ', foo.__defaults__)
foo()

Defaults:  ('Hello', 1)
Hello 1
Defaults:  ('Hello', 'world!')
Hello world!


#### Почему не стоит использовать mutable аргументы по умолчанию

In [ ]:
def foo(a, b=[]):
    b.append(a)
    print(*b)

In [ ]:
foo('Hello')
foo('the')
foo('wonderful')
foo('world!')
foo('world!')

Hello
Hello the
Hello the wonderful
Hello the wonderful world!
Hello the wonderful world! world!


In [ ]:
def foo(a):
    b=[]
    b.append(a)
    print(*b)

foo('Hello')
foo('the')
foo('wonderful')
foo('world!')

## @dataclass

Часто классы используются как контейнеры для данных. Для таких случаев удобно создавать классы с использованием `@dataclass` (Py >= 3.7) - удобного инструмента для создания классов, содержащих в себе большое количество атрибутов.

Посмотрим на примере класса `Person`. В обычном питоне он будет выглядеть так:

In [ ]:
class Person:
    def __init__(self, name, job, age):
        self.name = name
        self.job = job
        self.age = age

Какие минусы есть у такой реализации?

1. Не очень информативное представление:

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Можем исправить это с помощью `__str__` и `__repr__`.

In [ ]:
class Person:
    def __init__(self, name, job, age):
        self.name = name
        self.job = job
        self.age = age

    def __repr__(self):
        return f"Person: {self.name}, job: {self.job}, age: {self.age}"

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person: Geralt, job: Witcher, age: 30


2. Нет корректного сравнения "из коробки":

In [ ]:
person2 = Person("Yennefer", "Sorceress", 25)
person3 = Person("Yennefer", "Sorceress", 25)
print(person2 == person3)

False
137476458489616
137476458198880


Тоже можем починить - больше кода добавляется в наш класс!

In [ ]:
class Person:
    def __init__(self, name, job, age):
        self.name = name
        self.job = job
        self.age = age

    def __repr__(self):
        return f"Person(name='{self.name}', job='{self.job}', age={self.age})"

    def __eq__(self, other):
        return self.name == other.name and self.job == other.job and self.age == other.age

In [ ]:
person2 = Person("Yennefer", "Sorceress", 25)
person3 = Person("Yennefer", "Sorceress", 25)
print(person2 == person3)

True


Уже чувствуется боль... А мы даже не добавили проверок типов при инициализации...

Что предлагает `@dataclass`?

In [ ]:
from dataclasses import dataclass


@dataclass
class Person:
    name: str
    job: str
    age: int

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person(name='Geralt', job='Witcher', age=30)


In [ ]:
person2 = Person("Yennefer", "Sorceress", 25)
person3 = Person("Yennefer", "Sorceress", 25)
print(person2 == person3)

True


Вся "грязь" сделана за нас!

Что еще может датакласс?

### Сравнение и сортировка

In [ ]:
from dataclasses import dataclass


@dataclass(order=True)
class Person:
    sort_index: int   # Фиксированное имя
    name: str
    job: str
    age: int

    def __post_init__(self):
        self.sort_index = self.age

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
person2 = Person("Yennefer", "Sorceress", 25)
print(person1 > person2)

TypeError: ignored

Опаньки, теперь от нас требуется указывать 4 аргумента. Как подсказать dataclass, что sort_index не передается при инициализации?

In [ ]:
from dataclasses import dataclass, field


@dataclass(order=True)
class Person:
    sort_index: int = field(init=False)  # Подсказываем, что при инициации значение не нужно
    name: str
    job: str
    age: int

    def __post_init__(self):
        self.sort_index = self.age

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
person2 = Person("Yennefer", "Sorceress", 25)
print(person1 > person2)

True


Ура, победа!

In [ ]:
print(person1)

Person(sort_index=30, name='Geralt', job='Witcher', age=30)


Не совсем, `sort_index` при `print` нам тоже не нужен! Уберем:

In [ ]:
from dataclasses import dataclass, field


@dataclass(order=True)
class Person:
    sort_index: int = field(init=False, repr=False)
    name: str
    job: str
    age: int

    def __post_init__(self):
        self.sort_index = self.age

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person(name='Geralt', job='Witcher', age=30)


### Значения по умолчанию

Датаклассы поддерживают и это:

In [ ]:
from dataclasses import dataclass, field


@dataclass(order=True)
class Person:
    sort_index: int = field(init=False, repr=False)
    name: str
    job: str
    age: int
    strength: int = 100

    def __post_init__(self):
        self.sort_index = self.strength

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person(name='Geralt', job='Witcher', age=30, strength=100)


In [ ]:
hash(person1)

TypeError: ignored

А что делать, если хотим получать значение по умолчанию с помощью функции? Например, случайно генерировать:

In [ ]:
from dataclasses import dataclass, field
import random
import string


def generate_id():
    return "".join(random.choices(string.ascii_uppercase, k=12))


@dataclass(order=True)
class Person:
    sort_index: int = field(init=False, repr=False)
    name: str
    job: str
    age: int
    strength: int = 100
    id: str = field(default_factory=generate_id)

    def __post_init__(self):
        self.sort_index = self.strength

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person(name='Geralt', job='Witcher', age=30, strength=100, id='NOCYYSBLDVJU')


Если хотим убрать возможность убрать "ручное" присвоение id:

In [ ]:
from dataclasses import dataclass, field
import random
import string


def generate_id():
    return "".join(random.choices(string.ascii_uppercase, k=12))


@dataclass(order=True)
class Person:
    sort_index: int = field(init=False, repr=False)
    name: str
    job: str
    age: int
    strength: int = 100
    id: str = field(init=False, default_factory=generate_id)

    def __post_init__(self):
        self.sort_index = self.strength

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person(name='Geralt', job='Witcher', age=30, strength=100, id='HKLNJKNLTCAW')


А можем ли использовать как ключи словаря?

In [ ]:
hash(person1)

TypeError: ignored

### Объекты readonly

Можем заморозить объекты:

In [ ]:
from dataclasses import dataclass, field


@dataclass(order=True, frozen=True) # frozen - нельзя менять, как frozenset
class Person:
    sort_index: int = field(init=False, repr=False)
    name: str
    job: str
    age: int
    strength: int = 100

    def __post_init__(self):
        self.sort_index = self.strength

Проблема:

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

FrozenInstanceError: ignored

Как быть? Воспользоваться "обходным вариантом":

In [ ]:
from dataclasses import dataclass, field


@dataclass(order=True, frozen=True) # frozen - нельзя менять, как frozenset
class Person:
    sort_index: int = field(init=False, repr=False)
    name: str
    job: str
    age: int
    strength: int = 100

    def __post_init__(self):
        object.__setattr__(self, "sort_index", self.strength)

In [ ]:
person1 = Person("Geralt", "Witcher", 30)
print(person1)

Person(name='Geralt', job='Witcher', age=30, strength=100)


In [ ]:
person1.strength = 200

FrozenInstanceError: ignored

In [ ]:
hash(person1)

-5385388366469133182

## Метаклассы

В Python нет возможность перегружать методы обычного класса - вызывать по факту разные методы с одним и тем же именем в зависимости от типов получаемых аргументов.

Пример:

In [ ]:
class A:
    def f(self, x: int):
        print('A.f int overload', self, x)

    def f(self, x: str):
        print('A.f str overload', self, x)

    def f(self, x, y):
        print('A.f two arg overload', self, x, y)

В таком случае нижеидущее определение метода перетирает предыдущее:

In [ ]:
a = A()
a.f(1)

In [ ]:
a.f('1')

In [ ]:
a.f(1, 2)

Но перегрузку можно получить с помощью метаклассов!

Что такое **метаклассы**?

Каждый объект в Python имеет свой тип:

In [ ]:
print(f'{type(42)=}')
print(f'{type("hello")=}')
print(f'{type([])=}')
print(f'{type(a)=}')

type(42)=<class 'int'>
type("hello")=<class 'str'>
type([])=<class 'list'>
type(a)=<class '__main__.A'>


Классы позволяют создавать экземпляры себя. Но сами классы так же являются объектами, значит должны иметь свой тип:

In [ ]:
print(f'{type(int)=}')
print(f'{type(str)=}')
print(f'{type(list)=}')
print(f'{type(A)=}')

type(int)=<class 'type'>
type(str)=<class 'type'>
type(list)=<class 'type'>
type(A)=<class 'type'>


Тип каждого из этих классов - `type`.

Значит, так же, как можно создать "на лету" экземпляр класса `int` - число, можно создать экземпляр класса `type` - новый класс:

In [ ]:
x = int()
print(f"{x=}")

B = type('B', (), {})
print(f'{B=}')

x=0
B=<class '__main__.B'>


По сути, объявление класса через `class MyClass` - "синтаксический сахар" вокруг создания экземпляря `type`:

In [ ]:
def make_A():
    name = 'A'
    bases = ()  # от чего наследуемся

    a = 1
    b = 'hello'

    def f(self):
        return 42

    namespace = {'a': a, 'b': b, 'f': f}
    A = type(name, bases, namespace)
    return A

In [ ]:
A = make_A()
print(A)

a = A()
print(a.a, a.b, a.f())

<class '__main__.A'>
1 hello 42


Так где же здесь появляются метаклассы?

**Метаклассы** - наследники класса `type`, которые позволяют кастомизировать процесс создания новых классов.

Создадим простейший метакласс:

In [ ]:
class MyMetaclass(type):  # обязательно!
    pass

class A(metaclass=MyMetaclass):  # обязательно с ключевым словом!
    pass

In [ ]:
a = A()

print(f'{type(a)=}')
print(f'{type(A)=}')

type(a)=<class '__main__.A'>
type(A)=<class '__main__.MyMetaclass'>


Добавим функционал - будем считать время создания класса и сохранять в атрибут:

In [ ]:
import time

class LoadTimeMeta(type):
    base_time = time.perf_counter()

    def __new__(mcs, name, bases, namespace):
        print(mcs, name, bases, namespace)  # выводим аргументы, которые принимает __new__ для метаклассов
        namespace['__class_load_time__'] = time.perf_counter() - LoadTimeMeta.base_time  # добавляем атрибут
        return super().__new__(mcs, name, bases, namespace)  # делегируем процесс создания класса обратно type

class A(metaclass=LoadTimeMeta):
    pass

class B(A):
    pass

<class '__main__.LoadTimeMeta'> A () {'__module__': '__main__', '__qualname__': 'A'}
<class '__main__.LoadTimeMeta'> B (<class '__main__.A'>,) {'__module__': '__main__', '__qualname__': 'B'}


In [ ]:
print(f"{A.__class_load_time__=} after base time")
print(f"{B.__class_load_time__=} after base time")

A.__class_load_time__=0.005386454000017693 after base time
B.__class_load_time__=0.006714731000101892 after base time


>**Замечание** - метаклассы наследуются!

In [ ]:
print(f"{type(B)=}")

type(B)=<class '__main__.LoadTimeMeta'>


Где применяются метаклассы? При создании абстрактных базовых классов (ABC) -

In [ ]:
from abc import ABCMeta


class ABC(metaclass=ABCMeta):
    pass

class A(ABC):
    def __init__(self, *args, **kwargs):
        print("init", self, args, kwargs)

    @abstractmethod
    def f(self):
        pass

    @abstractmethod
    def g(self):
        pass

class B(A):
    def f(self):
        print('f!')

    def g(self):
        print('g!')

In [ ]:
a = A()

TypeError: ignored

In [ ]:
b = B()

init <__main__.B object at 0x7c0c3d126800> () {}
